# 선형 계층

## 행렬 곱

파이토치에서 행렬 곱을 구현해보자.

In [7]:
import torch
x = torch.FloatTensor([i for i in range(1, 7)]).reshape(3,2)
y = torch.FloatTensor([[1, 2], [1,2]])

print(x.size(), y.size())

torch.Size([3, 2]) torch.Size([2, 2])


파이토치의 matmul 함수를 이용하면 행렬 곱을 수행할 수 있다.

다음 코드는 앞서 만든 $x$와 $y$의 행렬 곱을 수행하고, 결과 행렬의 크기를 출력한다.

In [8]:
z = torch.matmul(x, y)
print(z.size())

torch.Size([3, 2])


### 배치 행렬 곱

딥러닝을 수행할 때 **보통 여러 샘플을 동시에 병렬 계산**하곤 한다. 따라서 행렬 곱 연산에도 여러 곱셈을 동시에 진행할 수 있어야 한다.

bmm 함수가 이 역할을 수행한다. 다음 코드와 같이 텐서를 선언해보자.

In [9]:
x = torch.FloatTensor(3,3,2)
y = torch.FloatTensor(3,2,3)

여기에서 bmm 함수를 활용하여 행렬 곱이 3번 수행되는 연산을 병렬로 동시에 진행할 수 있다.

In [10]:
z = torch.bmm(x, y)
print(z.size())

torch.Size([3, 3, 3])


bmm 함수는 마지막 2개의 차원을 행렬 취급하여 병렬로 행렬 곱 연산을 수행한다.

bmm 함수를 적용하기 위해서는 마지막 2개 차원을 제외한 다른 차원의 크기는 동일해야 한다. 즉, 배치의 크기가 같아야 한다.

## 선형 계층

데이터를 모아 어떠한 함수를 근사계산하고 싶을 때, 어떤 모델로 특정 함수를 근사계산할 수 있을까? 이 절에서는 그 기본 모델이 될 수 있는 **선형 계층**에 대해서 다뤄볼 것이다.

선형 계층은 심층신경망의 기본 구성 요소이고, 하나의 모델로도 동작할 수 있다.

또한 입력과 출력이 다르게 반환할 수도 있다. 예를 들어 4차원의 실수 벡터를 입력받아 3차원의 실수 벡터를 반환하는 함수로 볼 수 있다.

### 선형 계층 함수의 동작 방식

**가중치 파라미터**를 가지고 있으며, 이것에 의해 함수의 동작이 정의된다.

각각의 출력 노드의 값은 입력 노드로부터 들어오는 값에 가중치 파라미터 $W_{i \rightarrow j}$를 곱하고, 또 다른 가중치 파라미터 $b_{j}$를 더해서 결정한다.

입력층이 4개, 출력층이 3개라면 $W$에는 따라서 총 12개의 가중치 파라미터가 존재한다.

이 동작 방식은 다음과 같이 일반화 할 수 있다.

$$ y = f(x) = W^{T} \cdot x + b $$

수 백만 개의 입력 벡터가 주어졌을 때, 이를 단순히 순차적으로 처리한다면 매우 비효율 적이다. 따라서 이 연산을 다수의 입력을 처리하기 위한 병렬 연산으로 생각해볼 수도 있다.

$N$개의 $n$차원 벡터를 보아서 $N * n$의 행렬로 만들 수 있는데, 이것을 미니 배치라 한다.

선형 계층 함수에서 미니배치 행렬을 처리하기 위한 수식은 다음과 같다.

$$ y = f(x) = x \cdot W + b$$

입력을 $N$개 모아서 미니배치 행렬로 넣어주었기 때문에 출력도 $N$개의 $m$차원 벡터가 모여 $N * m$ 크기의 행렬이 된다.

이와 같은 병렬 계산을 통해 연산 속도를 높일 수 있다.

### 선형 계층의 의미

선형 계층은 **행렬 곱셈과 벡터의 덧셈으로 이루어져 있기 때문에 선형 변환**이라고 볼 수 이싿.

선형 계층을 통해 모델을 구성할 경우, 선형 데이터에 대한 관계를 분석하거나 선형 함수는 근사계산할 수 있다.

## 선형 계층 실습

### 직접 구현하기

선형 계층은 행렬 곱 연산과 브로드캐스팅 덧셈 연산으로 이루어져 있다.

선형 계층의 파라미터 행렬 $W$가 행렬 곱 연산에 활용될 것이고, 파라미터 벡터 $b$가 브로드캐스팅 덧셈 연산에 활용될 것이다.

In [14]:
W = torch.FloatTensor([i for i in range(1,7)]).reshape(3,2) # 가중치
b = torch.FloatTensor([2,2]) # 편향

def Linear(x, W,b):
    y = torch.matmul(x, W) + b
    return y

위와 같은 형식으로 선형 계층을 생성할 수 있다.

하지만 이 방법은 파이토치 입장에서 제대로 된 계층(layer)으로 취급하지 않는다.

제대로 된 계층을 만드는 방법을 지금부터 알아볼 것이다.

### torch.nn.Module 클래스 상속 받기

파이토치에는 nn(neural networks) 패키지가 있고, 내부에는 미리 정의된 많은 신경망들이 있다.

그리고 그 신경망들은 torch.nn.Module이라는 **추상 클래스를 상속**받아 정의되어 있다.

바로 이 추상 클래스를 상속받아 선형 계층을 구현할 수 있다.

In [15]:
import torch.nn as nn

nn.Module을 상속받은 MyLinear라는 메서드를 정의할 것이다.

nn.Module을 상속받은 클래스는 보통 2개의 method, __init__과 forward를 오버라이드한다(덮어쓴다).

    __init__ 함수는 계층 내부에서 필요한 변수를 미리 선언하고 있으며 또 다른 계층을 소유하도록 할 수 있다.

    forward 함수는 계층을 통과하는 데 필요한 계산 수행을 한다.

In [16]:
class MyLinear(nn.Module): # 추상 클래스 상속
    def __init__(self, input_dim = 3, output_dim = 2):
        self.input_dim = input_dim
        self.output_dim = output_dim

        super().__init__() # nn_Module 클래스를 호출한다. 우리는 nn을 활용해서 신경망을 구성할 것이기 때문에 super으로 부모 클래스를 호출하는 것이 중요.
        # 클래스를 인자로 넣을 때 자식클래스(부모클래스)로 인자를 삽입. 여기서 부모 클래스라는 것은 nn.Module이다. super을 통해 nn.Module의 인자들을 활용할 수 있다.
        # 오버라이딩은 자식 클래스와 부모 클래스의 함수가 겹칠 때 자식 클래스의 함수가 부모 클래스의 함수를 덮어 쓴다는 것.
        # 그렇다면 여기서의 super는 nn.Module의 함수들을 상속받되 __init__과 forward는 자식 클래스의 함수로 override하겠다는 의미로 해석할 수 있다.

        self.W = nn.Parameter(torch.FloatTensor(input_dim, output_dim)) # 임의의 가중치 W 생성
        self.b = nn.Parameter(torch.FloatTensor(output_dim)) # 임의의 가중치 b 생성

    def forward(self, x):
        y = torch.matmul(x, self.W) + self.b # W, b는 다음 함수에서 계속 써야하기 때문에 self로 지정
        return y

다음과 같이 파라미터가 정상적으로 출력되는 것을 볼 수 있다.

In [18]:
linear = MyLinear()

for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[-7.4488e-09,  2.0005e+00],
        [ 0.0000e+00, -2.5244e-29],
        [-2.0156e+00,  4.5730e-41]], requires_grad=True)
Parameter containing:
tensor([1.0489e-08, 1.2971e-11], requires_grad=True)


### nn.Linear 활용하기

torch.nn에 미리 정의된 선형 계층을 불러다 쓰면 간단하다. 다음 코드는 nn.Linear를 통해 선형 계층을 활용한 모습이다.

In [23]:
linear = nn.Linear(3,2)

x = torch.FloatTensor(1, 3)
y = linear(x)

for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[-0.4367,  0.1939,  0.2293],
        [-0.4986,  0.3862, -0.2407]], requires_grad=True)
Parameter containing:
tensor([ 0.0548, -0.1736], requires_grad=True)


nn.Module을 상속받아 정의한 MyLinear 클래스는 내부의 nn.Module 하위 클래스를 소유할 수 있다.

super을 통해 상속받았기 때문.

In [24]:
class MyLinear(nn.Module):
    def __init__(self, input_dim = 3, output_dim = 2):
        self.input_dim = input_dim
        self.output_dim = output_dim 

        super().__init__()

        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        y = self.linear(x)
        return y

nn.Module을 상속받아 MyLinear 클래스를 정의하고 있다.

__init__ 함수 내부에 nn.Linear을 선언, self.linear에 저장한다.

forward 함수에서는 self.linear에 텐서 $x$를 통과시킨다. 즉, 이 코드도 선형 계층을 구현한 것이라고 볼 수 있다.